In [1]:
from leo_utils import arc_point_on_earth, compute_satellite_intersection_point_enu, compute_az_el_dist
import numpy as np


# Generate RX positions

distances_km = [0.001]
azimuths_deg = np.linspace(0, 360, len(distances_km), endpoint=False)
gnd_positions = [np.array([0.0, 0.0, 0.0])]

for d_km, az in zip(distances_km, azimuths_deg):
    pos = arc_point_on_earth(d_km, az)
    gnd_positions.append(pos)
gnd_positions = np.array(gnd_positions)

for i, pos in enumerate(gnd_positions):
    print(f"TX{i}(m): {pos}")


# Compute SAT intersection points

sat_orbit_m = 550e3
angles = [(45, 90), (120, 80), (75,75),(80,88), (65, 85),(130,75)]
# angles = [(45, 90)]
sat_positions = []
delays_ms = []
fspl_db = []
frequency_hz = 10e9 
wavelength = 3e8 / frequency_hz

for az, el in angles:
    pos, delay, dist = compute_satellite_intersection_point_enu(az, el, sat_orbit_m)
    sat_positions.append(pos)
    delays_ms.append(delay)
    fspl = 20 * np.log10(4 * np.pi * dist / wavelength)
    fspl_db.append(fspl)

sat_positions = np.array(sat_positions)
delays_ms = np.array(delays_ms)
fspl_db = np.array(fspl_db)

print("\nSatellite Pos [m]:\n", sat_positions)
# print("\nPropagation delays [ms]:\n", delays_ms)
# print("\nFree-space path loss [dB]:\n", fspl_db)


# Compute az/el/dist per TX-SAT
  
for i, tx in enumerate(gnd_positions):
    print(f"\nFrom TX{i}:")
    for j, sat in enumerate(sat_positions):
        az, el, dist, n_waves = compute_az_el_dist(sat, tx, frequency_hz)
        print(f"  SAT{j}: az={az:.2f}°, el={el:.2f}°, dist={dist:.2f} m, λ count ≈ {n_waves:.2f}")


TX0(m): [0. 0. 0.]
TX1(m): [ 6.12323400e-17  1.00000000e+00 -7.85128629e-08]

Satellite Pos [m]:
 [[ 2.38137915e-11  2.38137915e-11  5.50000000e+05]
 [ 8.38834990e+04 -4.84301607e+04  5.49322180e+05]
 [ 1.41946635e+05  3.80344862e+04  5.48439683e+05]
 [ 1.89137181e+04  3.33499880e+03  5.49973353e+05]
 [ 4.35971558e+04  2.03296876e+04  5.49832825e+05]
 [ 1.12573272e+05 -9.44601913e+04  5.48439683e+05]]

From TX0:
  SAT0: az=45.00°, el=90.00°, dist=550000.00 m, λ count ≈ 18333333.33
  SAT1: az=120.00°, el=80.00°, dist=557796.36 m, λ count ≈ 18593212.01
  SAT2: az=75.00°, el=75.00°, dist=567786.54 m, λ count ≈ 18926218.02
  SAT3: az=80.00°, el=88.00°, dist=550308.59 m, λ count ≈ 18343619.53
  SAT4: az=65.00°, el=85.00°, dist=551933.10 m, λ count ≈ 18397769.90
  SAT5: az=130.00°, el=75.00°, dist=567786.54 m, λ count ≈ 18926218.02

From TX1:
  SAT0: az=180.00°, el=90.00°, dist=550000.00 m, λ count ≈ 18333333.33
  SAT1: az=120.00°, el=80.00°, dist=557796.45 m, λ count ≈ 18593214.90
  SAT2: a

In [2]:
from sionna.rt import Scene, Receiver, Transmitter, PlanarArray, PathSolver
import vsat_dish_3gpp
import numpy as np
import math

jam_rows = 32
jam_cols = 32
jam_antennas = jam_cols*jam_rows

sat_rows = 1
sat_cols = 1
sat_antennas = sat_cols*sat_rows

tx_rows = 32
tx_cols = 32
tx_antennas = tx_cols*tx_rows
# Create scene
scene = Scene()     

for tx_name in scene.transmitters:
    scene.remove(tx_name)
for rx_name in scene.receivers:
    scene.remove(rx_name)   
# Antenna pattern
scene.synthetic_array = True
scene.frequency = 10e9

tx_array = PlanarArray(num_rows=tx_rows ,
                        num_cols=tx_cols,
                        vertical_spacing=0.5,
                        horizontal_spacing=0.5,
                        pattern="tr38901",
                        polarization="V")

jam_array = PlanarArray(num_rows=jam_rows ,
                            num_cols=jam_cols,
                            vertical_spacing=0.5,
                            horizontal_spacing=0.5,
                        #  pattern="vsat_dish",
                            pattern="tr38901",
                            polarization="V")

sat_array = PlanarArray(num_rows=sat_rows ,
                             num_cols=sat_cols,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V")



# satellite as rx
scene.rx_array = sat_array
for i, pos in enumerate(sat_positions):
    # rx = Receiver(name=f"rx{i}", position=pos, orientation = [0, 180, 0])
    rx = Receiver(name=f"rx{i}", position=pos)
    scene.add(rx)

    rx.look_at(gnd_positions[0])
    
    
    



    
    

scene.tx_array = tx_array
tx = Transmitter(name=f"tx{0}", position=gnd_positions[0], display_radius=200)
scene.add(tx)
tx.look_at(sat_positions[0])
p_solver  = PathSolver()
paths = p_solver(scene=scene,  
                 max_depth=0,
                 los=True,
                 synthetic_array=True,
                 seed=41)

# CIR analysis
a_tx, tau_tx = paths.cir(normalize_delays=False, out_type="numpy")

for tx_name in scene.transmitters:
    scene.remove(tx_name)

scene.tx_array = jam_array
tx = Transmitter(name=f"tx{1}", position=gnd_positions[1], display_radius=200)
scene.add(tx)
# tx.look_at(sat_positions[0])
tx.look_at(sat_positions[0])


p_solver  = PathSolver()
paths = p_solver(scene=scene,
                 max_depth=0,
                 los=True,
                 synthetic_array=True,
                 seed=41)


a_jam, tau_jam = paths.cir(normalize_delays=False, out_type="numpy")

H0 = a_tx[:, 0, 0, :, 0, 0]  
H1 = a_jam[:, 0, 0, :, 0, 0]  





In [3]:
import numpy as np
import cvxpy as cp

def compute_beamforming_vector(H, mode="eig1"):
    """
    Compute the transmit beamforming vector using one of:
    - 'eig1': Eigen-decomposition on unweighted Q
    - 'eig2': Eigen-decomposition on weighted Q (equalizes TX energy per direction)
    - 'avg' : Normalized average of effective channels
    - 'fair': Max-min fairness (equalize RX received energy)

    Parameters:
        H: numpy array of shape (N_rx, rx_antennas, N_tx, tx_antennas)
        mode: 'eig1', 'eig2', 'avg', or 'fair'

    Returns:
        w: Normalized beamforming vector of shape (tx_antennas, 1)
        norm_last_h_eff: Norm of last h_eff computed (debug/info)
    """
    N_rx, rx_antennas, N_tx, tx_antennas = H.shape
    assert N_tx == 1, "Only supports single TX (N_tx == 1)"

    w_r = np.ones((rx_antennas, 1), dtype=complex) / np.sqrt(rx_antennas)

    if mode in ("eig1", "eig2"):
        Q = np.zeros((tx_antennas, tx_antennas), dtype=complex)
        for rx in range(N_rx):
            H_rx = H[rx, :, 0, :]           # (rx_antennas, tx_antennas)
            h_eff = H_rx.conj().T @ w_r     # (tx_antennas, 1)
            alpha = 1.0 / np.linalg.norm(h_eff)**2 if mode == "eig2" else 1.0
            Q += alpha * (h_eff @ h_eff.conj().T)
        eigvals, eigvecs = np.linalg.eigh(Q)
        w = eigvecs[:, -1]

    elif mode == "avg":
        w_sum = np.zeros((tx_antennas, 1), dtype=complex)
        for rx in range(N_rx):
            H_rx = H[rx, :, 0, :]
            h_eff = H_rx.conj().T @ w_r
            h_eff_unit = h_eff / np.linalg.norm(h_eff)
            w_sum += h_eff_unit
        w = w_sum[:, 0]
        w /= np.linalg.norm(w)
    elif mode == "fair":
        w_var = cp.Variable((tx_antennas, 1), complex=True)
        t = cp.Variable()
        constraints = [cp.norm(w_var) <= 1]

        for rx in range(N_rx):
            H_rx = H[rx, :, 0, :]              # (M, N)
            h_eff = H_rx.conj().T @ w_r        # (N, 1)
            A = h_eff @ h_eff.conj().T
            A = (A + A.conj().T) / 2           # Force Hermitian
            expr = cp.real(cp.matmul(cp.matmul(w_var.H, A), w_var))[0, 0]
            constraints.append(expr >= t)

        problem = cp.Problem(cp.Maximize(t), constraints)
        problem.solve()
        w = w_var.value[:, 0]


    else:
        raise ValueError("mode must be 'eig1', 'eig2', 'avg', or 'fair'")

    w = w.reshape(-1, 1)
    return w, np.linalg.norm(h_eff)


In [4]:
def compute_rx_gains(H, w):
    """
    Compute beamforming gains at each RX.

    Parameters:
        H: shape (N_rx, rx_antennas, 1, tx_antennas)
        w: beamforming vector of shape (tx_antennas, 1)

    Returns:
        gains: numpy array of shape (N_rx,), each entry is a gain (scalar)
    """
    N_rx, rx_antennas, _, tx_antennas = H.shape
    w_r = np.ones((rx_antennas, 1)) / np.sqrt(rx_antennas)
    gains = np.zeros(N_rx)

    for rx in range(N_rx):
        H_rx = H[rx, :, 0, :]                       # (rx_antennas, tx_antennas)
        h_combined = w_r.conj().T @ H_rx @ w        # scalar (1×M) · (M×N) · (N×1)
        gains[rx] = np.abs(h_combined[0, 0])**2     # real-valued scalar

    return gains

In [5]:
H_jam = a_jam[..., 0, 0]   
w1,h1 = compute_beamforming_vector(H_jam, mode="eig1")  # unweighted eig
w2,h2= compute_beamforming_vector(H_jam, mode="eig2")  # weighted eig
w3 ,h3= compute_beamforming_vector(H_jam, mode="avg")   # avg sum
# w4 ,_= compute_beamforming_vector(H_jam, mode="fair")   # 
w0 = np.ones((jam_antennas, 1)) / np.sqrt(jam_antennas)
jam_gains0 = compute_rx_gains(H_jam, w0)
jam_gains1 = compute_rx_gains(H_jam, w1)
jam_gains2 = compute_rx_gains(H_jam, w2)
jam_gains3 = compute_rx_gains(H_jam, w3)
# print(jam_gains0)
# print(jam_gains1)
# print(jam_gains2)  
# print(jam_gains3)  

In [6]:

H_tx = a_tx[..., 0, 0]             
w_tx1, h_tx1 = compute_beamforming_vector(H_tx, mode="eig1")
w_tx2, h_tx2 = compute_beamforming_vector(H_tx, mode="eig2")
w_tx3, h_tx3 = compute_beamforming_vector(H_tx, mode="avg")
w_tx0 = np.ones((tx_antennas, 1)) / np.sqrt(jam_antennas)
tx_gains0 = compute_rx_gains(H_tx, w_tx0)
tx_gains1 = compute_rx_gains(H_tx, w1)
tx_gains2 = compute_rx_gains(H_tx, w2)
tx_gains3 = compute_rx_gains(H_tx, w3)

In [7]:
c = 3e8  # speed of light (m/s)
fc = 10e9  # Carrier frequency: 10 GHz
wavelength = c / fc
bandwidth = 100e6  # 100 MHz
tx_power_dbm = 30  #  dBm
jam_power_dbm = 50
k = 1.38e-23  # Boltzmann 
GT = 13 # db gain-to-noise-temperature for for 0.33m Equivalent satellite antenna aperturesatellites, or can be 5 dB K^(-1) for 0.13m Equivalent satellite antenna aperture
La = 5 # dB
GT_linear_inv = 10 ** (-GT / 10)
La_linear = 10 ** (La / 10)
noise_power_watt = k * bandwidth * GT_linear_inv * La_linear
Tx_power_watt = 10 ** ((tx_power_dbm  - 30)/ 10)  
jam_power_watt = 10 ** ((jam_power_dbm  - 30)/ 10)


In [8]:
def compute_sinr_db(Tx_power_watt,jam_power_watt, jam_gain_linear,tx_gain_linear,noise_power_watt):

    return 10 * np.log10((Tx_power_watt * tx_gain_linear) / (jam_power_watt * jam_gain_linear + noise_power_watt))
    # return 10 * np.log10((Tx_power_watt * tx_gain_linear) / ( noise_power_watt))


sinr0 = compute_sinr_db(tx_power_dbm,jam_power_dbm, jam_gains0,tx_gains0,noise_power_watt)
sinr1 = compute_sinr_db(tx_power_dbm,jam_power_dbm, jam_gains1,tx_gains1,noise_power_watt)
sinr2 = compute_sinr_db(tx_power_dbm,jam_power_dbm, jam_gains2,tx_gains2,noise_power_watt)
sinr3 = compute_sinr_db(tx_power_dbm,jam_power_dbm, jam_gains3,tx_gains3,noise_power_watt)


In [9]:
import pandas as pd
import numpy as np

sinr_matrix = np.vstack([sinr0, sinr1, sinr2, sinr3])


df = pd.DataFrame(
    sinr_matrix,
    index=["w0_none", "w1_eig1", "w2_eig2", "w3_avg"],   
    columns=[f"Sat{i}" for i in range(1, 7)]            
)

print(df.round(2).to_string())


         Sat1   Sat2   Sat3   Sat4   Sat5   Sat6
w0_none -8.27  -6.88 -10.57  11.49   7.50 -13.52
w1_eig1 -8.27 -11.15 -10.16  11.17   4.75 -15.40
w2_eig2 -8.27 -13.68  -4.76   3.20  -5.23 -11.14
w3_avg  -8.27 -39.05 -33.30   5.42 -12.44 -43.65


In [10]:
import numpy as np

def combine_sinr_db(sinr_db):
    sinr_lin = 10**(sinr_db / 10)        
    total_lin = sinr_lin.sum()          
    return 10 * np.log10(total_lin)      

comb0 = combine_sinr_db(sinr0)
comb1 = combine_sinr_db(sinr1)
comb2 = combine_sinr_db(sinr2)
comb3 = combine_sinr_db(sinr3)

print(f"combined SINR0 = {comb0:.2f} dB")
print(f"combined SINR1 = {comb1:.2f} dB")
print(f"combined SINR2 = {comb2:.2f} dB")
print(f"combined SINR3 = {comb3:.2f} dB")


combined SINR0 = 13.05 dB
combined SINR1 = 12.15 dB
combined SINR2 = 4.76 dB
combined SINR3 = 5.67 dB


I want to add that if the jammer and the desired source are within the diffraction limits of the receiver aperture, then we are in bigger trouble as you cannot use beamforming to null out the jammer. Then nonlinear subtraction methods may have to be used and that will hit the receiver sensitivity a lot I a sure. But without it there is no hope as both signal and jammer are smack on the beam.